In [24]:
import jax
import jax.numpy as jnp

win_length = 5

ones = jnp.ones((1, win_length), dtype=jnp.int8)  # shape: (1, win_length)
zeros = jnp.zeros((win_length-1, win_length), dtype=jnp.int8)  # shape: (win_length-1, win_length)

# Create horizontal kernel
horizontal = jnp.expand_dims(jnp.vstack([ones, zeros]), axis=(0,1))  # shape: (1, 1, win_length, win_length)
vertical = jnp.expand_dims(jnp.vstack([ones, zeros]).T, axis=(0,1))  # shape: (1, 1, win_length, win_length)
diagonal = jnp.expand_dims(jnp.eye(win_length, dtype=jnp.int8), axis=(0, 1))  # shape: (1, 1, win_length, win_length)
anti_diagonal = jnp.expand_dims(jnp.fliplr(jnp.eye(win_length, dtype=jnp.int8)), axis=(0, 1))  # shape: (1, 1, win_length, win_length)

# Stack all kernels


In [47]:
    def _check_win(self):
        """
        Checks if the current board state has a winning condition.
        Uses convolution with predefined kernels.
        Returns a JAX boolean (without using .item()).
        """
        # Add necessary dimensions to current_player for broadcasting
        current_player_reshaped = self.current_player[:, jnp.newaxis, jnp.newaxis]
        player_boards = (self.board * current_player_reshaped)[:, :, :, jnp.newaxis]
        padding = ((WIN_LENGTH - 1, WIN_LENGTH - 1), (WIN_LENGTH - 1, WIN_LENGTH - 1))

        conv_output = lax.conv_general_dilated(
            player_boards,
            self.kernels,
            window_strides=(1, 1),
            padding=padding,
            dimension_numbers=('NHWC', 'HWIO', 'NHWC'),
        )
        win_condition = conv_output == WIN_LENGTH
        win = jnp.any(win_condition)&~self.dones
        return win  # Do not call .item(), just return the traced value




(1, 6, 6, 4)
True


In [35]:
print(board.shape)
print(kernels.shape)

(1, 10, 10)
(5, 5, 4)


In [36]:
def get_valid_actions(board):
    return board == 0

print(get_valid_actions(board))

87


In [46]:
from jax import random
arr = random.normal(random.PRNGKey(0), shape=(10,10))

print(arr[None,:,None,:,None].shape)

(1, 10, 1, 10, 1)


In [48]:
    def _check_win(self):
        """
        Checks if the current board state has a winning condition.
        Uses convolution with predefined kernels.
        Returns a JAX boolean (without using .item()).
        """
        # Add necessary dimensions to current_player for broadcasting
        current_player_reshaped = self.current_player[:, jnp.newaxis, jnp.newaxis]
        player_boards = (self.board * current_player_reshaped)[:, :, :, jnp.newaxis]
        padding = ((WIN_LENGTH - 1, WIN_LENGTH - 1), (WIN_LENGTH - 1, WIN_LENGTH - 1))

        conv_output = lax.conv_general_dilated(
            player_boards,
            self.kernels,
            window_strides=(1, 1),
            padding=padding,
            dimension_numbers=('NHWC', 'HWIO', 'NHWC'),
        )
        win_condition = conv_output == WIN_LENGTH
        win = jnp.any(win_condition)&~self.dones
        return win  # Do not call .item(), just return the traced value



In [ ]:
game = GomokuEnv(num_envs=20)

for i in range(100):
    action_mask = game.get_action_mask()
    

In [7]:
from jax import lax
import jax.numpy as jnp
WIN_LENGTH = 5
def _create_kernels():
    """
    Creates the kernels to check win conditions.
    """
    ones = jnp.ones((1, WIN_LENGTH), dtype=jnp.float32)  # shape: (1, win_length)
    zeros = jnp.zeros((WIN_LENGTH-1, WIN_LENGTH), dtype=jnp.float32)  # shape: (win_length-1, win_length)

    horizontal = jnp.expand_dims(jnp.vstack([ones, zeros]), axis=(0,1))  # shape: (1, 1, win_length, win_length)
    vertical = jnp.expand_dims(jnp.vstack([ones, zeros]).T, axis=(0,1))  # shape: (1, 1, win_length, win_length)
    diagonal = jnp.expand_dims(jnp.eye(WIN_LENGTH, dtype=jnp.float32), axis=(0, 1))  # shape: (1, 1, win_length, win_length)
    anti_diagonal = jnp.expand_dims(jnp.fliplr(jnp.eye(WIN_LENGTH, dtype=jnp.float32)), axis=(0, 1))  # shape: (1, 1, win_length, win_length)

    # Stack all kernels
    kernels = jnp.concatenate([horizontal, vertical, diagonal, anti_diagonal], axis=0).transpose(2,3,1,0)
    return kernels  # Shape: (win_length, win_length, 1, 4)


In [42]:
from jax import lax
WIN_LENGTH = 5


def _check_win(board,kernels):
    """
    Checks if placing a stone at (row, col) leads to a win for the given player.
    A win is defined as having 5 or more consecutive stones in any direction.
    """
    # Prepare the board for convolution
    # Shape: (num_envs, board_size, board_size, 1)

    # Perform convolution
    conv_output = lax.conv_general_dilated(
        board,
        kernels,
        window_strides=(1, 1),
        padding='SAME',
        dimension_numbers=('NHWC', 'HWIO', 'NHWC'),
        feature_group_count=1
    )  # Shape: (active_boards_shape, board_size, board_size,4)

    win_condition = conv_output == WIN_LENGTH
    print(win_condition.shape)
    winners = jnp.any(win_condition)
    return winners.item()


In [43]:
kernels = _create_kernels()
print(kernels.shape)

(5, 5, 1, 4)


In [44]:
import jax
board = jax.random.randint(key=jax.random.PRNGKey(0),shape=(10,10),minval=0,maxval=2).astype(jnp.float32)
boards = board[jnp.newaxis,:,:,jnp.newaxis]
_check_win(boards,kernels)

(1, 10, 10, 4)


True

In [5]:
import jax.numpy as jnp

arr =  jnp.array([[True,True],[True,True]])

if jnp.all(arr):
    print("all true")
else:
    print("not all true")

all true


In [8]:
import jax
jax.tree.map(lambda x, y: [x] + y, [5, 6,8], [[7, 9], [1, 2],[0]])

[[5, 7, 9], [6, 1, 2], [8, 0]]

In [12]:
WIN_LENGTH = 5
def _create_kernels():
    ones = jnp.ones((1, WIN_LENGTH), dtype=jnp.float32)
    zeros = jnp.zeros((WIN_LENGTH - 1, WIN_LENGTH), dtype=jnp.float32)

    horizontal = jnp.expand_dims(jnp.vstack([ones, zeros]), axis=(0, 1))
    vertical = jnp.expand_dims(jnp.vstack([ones, zeros]).T, axis=(0, 1))
    diagonal = jnp.expand_dims(jnp.eye(WIN_LENGTH, dtype=jnp.float32), axis=(0, 1))
    anti_diagonal = jnp.expand_dims(jnp.fliplr(jnp.eye(WIN_LENGTH, dtype=jnp.float32)), axis=(0, 1))

    kernels = jnp.concatenate([horizontal, vertical, diagonal, anti_diagonal], axis=0).transpose(2, 3, 1, 0)
    return kernels

def _check_win(board):
    """
    Checks if the current board state has a winning condition.
    Uses convolution with predefined kernels.
    Returns a JAX boolean (without using .item()).
    """
    # Add necessary dimensions to current_player for broadcasting
    player_boards = (board)[:, :, :, jnp.newaxis]
    padding = ((WIN_LENGTH - 1, WIN_LENGTH - 1), (WIN_LENGTH - 1, WIN_LENGTH - 1))

    conv_output = lax.conv_general_dilated(
        player_boards,
        kernels,
        window_strides=(1, 1),
        padding=padding,
        dimension_numbers=('NHWC', 'HWIO', 'NHWC'),
    )
    print(conv_output.shape)
    win_condition = abs(conv_output) == WIN_LENGTH
    win = jnp.any(win_condition,axis=(-3,-2,-1))
    return win # Do not call .item(), just return the traced value






In [13]:
import jax.numpy as jnp
from jax import lax


# All three boards stacked in a single array with shape (3, 10, 10)
boards = jnp.array([
    # Board 0
    [
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, -1, -1, -1, -1, -1, 0, 0, 0],
        [0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    ],
    
    # Board 1
    [
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, -1, 0, 0, 0],
        [0, 0, 1, 0, -1, 0, 0, -1, 0, 0],
        [0, 0, 0, 0, 0, -1, 0, 0, -1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    ],
    
    # Board 2
    [
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, -1, -1, -1, -1, -1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    ]
], dtype=jnp.float32)

print(boards.shape)
kernels = _create_kernels()
print(_check_win(boards))

(3, 10, 10)
(3, 14, 14, 4)
[ True False  True]


In [6]:
print(row)
print(col)

NameError: name 'row' is not defined

In [35]:
board = jnp.zeros((3,15,15))
board = board.at[jnp.arange(3),row,col].set(1)

In [2]:
import jax.numpy as jnp
a = (1,2,3)
b = (4,5,6)

print(jnp.array(list(zip(a,b))))

[[1 4]
 [2 5]
 [3 6]]


In [6]:
c  = jnp.array(list(zip(a,b))).T


In [7]:
print(c[0]*3+c[1])

[ 7 11 15]


In [16]:
import jax
import jax.numpy as jnp

def preprocess_trajectory(trajectory):
    """
    Processes a batched trajectory from multiple parallel environments whose episodes
    might end at different time steps. For each environment, the trajectory is truncated
    at the first time step where the done flag becomes True. The valid data from all
    environments are concatenated along the time dimension.

    Args:
        trajectory: dict containing keys:
            "obs": jnp.ndarray with shape (T, num_envs, ...) e.g. (T, num_envs, board_size, board_size)
            "actions": jnp.ndarray with shape (T, num_envs, 2)
            "log_probs": jnp.ndarray with shape (T, num_envs)
            "rewards": jnp.ndarray with shape (T, num_envs)
            "dones": jnp.ndarray with shape (T, num_envs), where True indicates the episode is done.

    Returns:
        A dict with keys "obs", "actions", "log_probs", and "rewards" containing the valid,
        concatenated trajectory data from all environments.
    """
    # Make sure rewards and dones are JAX arrays.
    trajectory["rewards"] = jnp.array(trajectory["rewards"])
    trajectory["dones"] = jnp.array(trajectory["dones"])

    T, num_envs = trajectory["obs"].shape[0], trajectory["obs"].shape[1]
    indices = jnp.argmax(trajectory["dones"], axis=0)
    mask = jnp.arange(T)[:, None] <= indices[None, :]
    valid_obs = trajectory["obs"][mask]
    valid_actions = trajectory["actions"][mask]
    valid_log_probs = trajectory["log_probs"][mask]
    valid_rewards = trajectory["rewards"][mask]

    return {
        "obs": valid_obs,
        "actions": valid_actions,
        "log_probs": valid_log_probs,
        "rewards": valid_rewards
    }

def main():
    # Define trajectory dimensions.
    T = 6         # number of time steps
    num_envs = 3  # number of parallel environments

    # Create fake data for trajectory.
    # For simplicity, observations will be unique integers reshaped to (T, num_envs, 1)
    obs = jnp.arange(T * num_envs).reshape(T, num_envs, 1)
    
    # Actions: Create two columns per action (e.g. row, col)
    actions_data = jnp.arange(T * num_envs).reshape(T, num_envs, 1)
    actions = jnp.concatenate([actions_data, actions_data + 100], axis=-1)

    # Log probabilities (for example, a linear spacing between 0 and 1)
    log_probs = jnp.linspace(0, 1, T * num_envs).reshape(T, num_envs)

    # Rewards (also a linear spacing for illustration)
    rewards = jnp.linspace(0, 1, T * num_envs).reshape(T, num_envs)

    # dones: simulate each environment ending at a different time.
    # Environment 0: done at step 2, Environment 1: done at step 5, Environment 2: done at step 3
    dones = jnp.array([
        [False, False, False],  # step 0: all active
        [False, False, False],  # step 1
        [True,  False, False],  # step 2: env 0 ends
        [True, False, True ],  # step 3: env 2 ends
        [True, False, True ],  # step 4
        [True, True,  True]   # step 5: env 1 ends
    ])

    # Build trajectory dictionary.
    trajectory = {
        "obs": obs,           # shape (T, num_envs, 1)
        "actions": actions,   # shape (T, num_envs, 2)
        "log_probs": log_probs,   # shape (T, num_envs)
        "rewards": rewards,       # shape (T, num_envs)
        "dones": dones            # shape (T, num_envs)
    }

    print("Original trajectory shapes:")
    print("obs:", trajectory["obs"].shape)
    print("actions:", trajectory["actions"].shape)
    print("log_probs:", trajectory["log_probs"].shape)
    print("rewards:", trajectory["rewards"].shape)
    print("dones:", trajectory["dones"].shape)

    # Process the trajectory using our function.
    processed = preprocess_trajectory(trajectory)

    # Print results.
    print("\nProcessed trajectory:")
    print("Processed Observations (shape {}):".format(processed["obs"].shape))
    print(processed["obs"])
    print("Processed Actions (shape {}):".format(processed["actions"].shape))
    print(processed["actions"])
    print("Processed Log Probs (shape {}):".format(processed["log_probs"].shape))
    print(processed["log_probs"])
    print("Processed Rewards (shape {}):".format(processed["rewards"].shape))
    print(processed["rewards"])

if __name__ == "__main__":
    main()

Original trajectory shapes:
obs: (6, 3, 1)
actions: (6, 3, 2)
log_probs: (6, 3)
rewards: (6, 3)
dones: (6, 3)

Processed trajectory:
Processed Observations (shape (13, 1)):
[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [10]
 [11]
 [13]
 [16]]
Processed Actions (shape (13, 2)):
[[  0 100]
 [  1 101]
 [  2 102]
 [  3 103]
 [  4 104]
 [  5 105]
 [  6 106]
 [  7 107]
 [  8 108]
 [ 10 110]
 [ 11 111]
 [ 13 113]
 [ 16 116]]
Processed Log Probs (shape (13,)):
[0.         0.05882353 0.11764706 0.1764706  0.23529412 0.29411766
 0.3529412  0.4117647  0.47058824 0.5882353  0.64705884 0.7647059
 0.9411765 ]
Processed Rewards (shape (13,)):
[0.         0.05882353 0.11764706 0.1764706  0.23529412 0.29411766
 0.3529412  0.4117647  0.47058824 0.5882353  0.64705884 0.7647059
 0.9411765 ]


In [19]:
import jax.numpy as jnp
action = jnp.array([[1,2],[3,4],[5,6]])
rows, cols = action.T

    # Create a boolean mask: True for boards that are not done.
done = jnp.array([False,False,True])
active_mask = jnp.logical_not(done)
# Get the indices for active boards.
active_idx = jnp.where(active_mask)

In [22]:
print(type(active_idx))

<class 'tuple'>


In [24]:
def sample_action(logits, rng):
        """
        Samples an action from the given logits and returns a tuple (row, col).

        Args:
            logits (jnp.ndarray): A tensor of shape (batch, board_size, board_size)
                                  representing the unnormalized log-probabilities.
            rng: A JAX PRNGKey for randomness.

        Returns:
            jnp.ndarray: Selected moves as an array with shape (2, batch).
        """
        batch_size, board_size, _ = logits.shape
        flat_logits = logits.reshape((batch_size, board_size * board_size))
        flat_actions = jax.random.categorical(rng, flat_logits, axis=-1)
        rows = flat_actions // board_size
        cols = flat_actions % board_size
        return jnp.array(list(zip(rows, cols)))


In [30]:
batch_size = 10
board_size = 15
arr = jnp.array([-jnp.inf]*batch_size*board_size*board_size).reshape(batch_size,board_size,board_size)
rng = jax.random.PRNGKey(12131232)
action = sample_action(arr,rng)

In [67]:
def discount_rewards(rewards, gamma):
    """
    Calculate discounted rewards for batched trajectories.

    Args:
      rewards: a 2D jnp.array with shape (batch, T)
      gamma: discount factor.
      
    Returns:
      A jnp.array of discounted rewards with the same shape as `rewards`.
    """
    def discount_single(r):
      def scan_fn(carry, reward):
        new_carry = reward + gamma * carry
        return new_carry, new_carry

      _, discounted_reversed = lax.scan(scan_fn, 0.0, r[::-1])
      return discounted_reversed[::-1]
    return jax.vmap(discount_single,in_axes=1,out_axes=1)(rewards)


In [68]:
rewards = jnp.array([[1,2,3,4,5],[1,2,3,4,5]])
discounted_rewards = discount_rewards(rewards,0.99)
print(discounted_rewards)

[[1.99      3.98      5.9700003 7.96      9.95     ]
 [1.        2.        3.        4.        5.       ]]


In [5]:
import numpy as np
nx, ny = (5, 5)
x = np.arange(nx)
print(x)
y = np.arange(ny)
print(y)
xv, yv = np.meshgrid(x, y)
print(xv)
print(yv)

[0 1 2 3 4]
[0 1 2 3 4]
[[0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]]
[[0 0 0 0 0]
 [1 1 1 1 1]
 [2 2 2 2 2]
 [3 3 3 3 3]
 [4 4 4 4 4]]
